<h1>Ex 3 week3</h1>
<p>
    Added geolocation, in this notebook we cluster and display on Toronto noeighbourhoods.
  </p>
  
  <br>
  <h4>Create dataframe (ex1)</h4>

In [2]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
import requests 
from bs4 import BeautifulSoup
#get wiki page content
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page
#get table content using selector '#mw-content-text > div > table.wikitable.sortable.jquery-tablesorter > tbody'
soup = BeautifulSoup(page.content, "lxml")

tables = soup.findChildren('table')
rows = tables[0].findChildren(['tr'])

df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])
for tr in tables[0].find_all('tr'):
    #  
    tds = tr.select('td')
    # 
    if tds and tds[1].get_text() != 'Not assigned':
        neighbour = (tds[2].get_text().rstrip())  if (tds[2].get_text().rstrip() != 'Not assigned') else (tds[1].get_text().rstrip())
        
        if (len(df[df['PostalCode']==tds[0].get_text()]) == 0):
            df = df.append({'PostalCode':tds[0].get_text(), 'Borough':tds[1].get_text(), 'Neighbourhood':neighbour}, ignore_index=True)   
        else:           
            df.loc[df['PostalCode'] == tds[0].get_text(), 'Neighbourhood'] += ', '+neighbour;
        
print(df.shape)



(103, 3)


  <h4>Add latitud and longitude dataframe (ex2)</h2>

In [6]:
#Geospatial data is get from csv file and setted index to use to compose latitude and longitud using postalcode
file_name='Geospatial_Coordinates.csv'
df_coorindates=pd.read_csv(file_name)
df_coorindates.head()
df_coorindates.set_index('Postal Code', inplace=True)

#use lambda function to create new colum using postal code as index (row.name)
df['Latitude']= df.apply(lambda row: df_coorindates.loc[df['PostalCode']]['Latitude'][row.name] , axis = 1)
df['Longitude']= df.apply(lambda row: df_coorindates.loc[df['PostalCode']]['Longitude'][row.name] , axis = 1)


<h2>Exercice 3 stats here</h2>

In [7]:
from sklearn.preprocessing import StandardScaler

#dataset normalization
X = df.values[:,3:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)


In [8]:
# K Means training

num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
clustered = k_means.labels_

df['Clusters'] = clustered

# Cluster Values is added to the dataframe
print(df.head())

  PostalCode           Borough                     Neighbourhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto         Harbourfront, Regent Park  43.654260   
3        M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4        M7A      Queen's Park                      Queen's Park  43.662301   

   Longitude  Clusters  
0 -79.329656         0  
1 -79.315572         0  
2 -79.360636         2  
3 -79.464763         1  
4 -79.389494         2  


In [9]:
#Folium import
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.



In [10]:
# assigning Toronto latitude and longitude values:
latitude=43.6532
longitude=-79.3832

#colors to print different clusters
clustercolor = ['red', 'green', 'black']

# create map of Totonto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood, clustered in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['Clusters']):
    label = folium.Popup(neighbourhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='yellow',
        fill=True,
        fill_color=clustercolor[clustered],
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto